<a href="https://colab.research.google.com/github/paarth-9k/DSA/blob/main/ibm_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Part 1: Install the necessary libraries
!pip install qiskit qiskit-aer qiskit-ibm-runtime matplotlib pylatexenc

# Part 2: Import and Run on Real IBM Hardware
from qiskit_ibm_runtime import QiskitRuntimeService
# The Sampler for the cloud is now called SamplerV2
from qiskit_ibm_runtime.sampler import SamplerV2 as Sampler
from qiskit import QuantumCircuit, transpile

print("\n--- Libraries installed and imported successfully ---\n")

# --- Your function to create the circuit ---
def create_bv_circuit(secret_string):
    num_qubits = len(secret_string) + 1
    qc = QuantumCircuit(num_qubits, len(secret_string))
    qc.h(range(num_qubits-1)); qc.x(num_qubits-1); qc.h(num_qubits-1); qc.barrier()
    for i, char in enumerate(reversed(secret_string)):
        if char == '1': qc.cx(i, num_qubits-1)
    qc.barrier(); qc.h(range(num_qubits-1)); qc.barrier()
    qc.measure(range(len(secret_string)), range(len(secret_string)))
    return qc

secret_key = '101'
victim_circuit = create_bv_circuit(secret_key)
# ---------------------------------------------

# Step 1: Connect to the IBM Quantum service
service = QiskitRuntimeService(
    channel="ibm_cloud",
    token="KYLgcVygZxhhdMlHPbHXc71p64PE-Wgb1C7So3Qbfd_w",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/c4b54a6a008d47f2afbe785476cdbd53:93ed1496-2c52-4df5-b0db-43c824e7a6b2::"
)
backend_real = service.backend("ibm_brisbane")
print(f"Successfully connected to backend: {backend_real.name}")

# Step 2: Transpile the circuit for the target backend
print("\nTranspiling circuit for the target backend...")
transpiled_circuit = transpile(victim_circuit, backend_real)
print("Circuit transpiled successfully.")

# Step 3: Initialize the SamplerV2 with the backend object
# The free plan is handled automatically by this new interface
sampler = Sampler(mode=backend_real)
shots = 8192

# Step 4: Run the baseline job using the sampler
print("\nSubmitting BASELINE job to real hardware via SamplerV2...")
# The transpiled circuit must be passed inside a list
job_baseline = sampler.run([transpiled_circuit], shots=shots)
print(f"Job submitted with ID: {job_baseline.job_id()}")
print("Waiting for job to complete (this may take several minutes)...")

result_baseline = job_baseline.result()

# Step 5: Correctly process the new SamplerV2 result format
pub_result = result_baseline[0]
counts = pub_result.data.c.get_counts()
prob_dist_baseline = {bitstring: count / shots for bitstring, count in counts.items()}


print("\n✅ Baseline job finished!")
print(f"Baseline results (probabilities): {prob_dist_baseline}")


# Step 6: Calculate the baseline success probability
if secret_key in prob_dist_baseline:
    p_success_baseline = prob_dist_baseline[secret_key]
else:
    p_success_baseline = 0

print(f"\nBaseline Success Probability: {p_success_baseline:.4f}")


--- Libraries installed and imported successfully ---

Successfully connected to backend: ibm_brisbane

Transpiling circuit for the target backend...
Circuit transpiled successfully.

Submitting BASELINE job to real hardware via SamplerV2...
Job submitted with ID: d2ulfqrok8rs73ct9ot0
Waiting for job to complete (this may take several minutes)...

✅ Baseline job finished!
Baseline results (probabilities): {'101': 0.906494140625, '000': 0.0194091796875, '111': 0.0201416015625, '001': 0.0330810546875, '100': 0.0191650390625, '110': 0.00048828125, '011': 0.0009765625, '010': 0.000244140625}

Baseline Success Probability: 0.9065


In [19]:
# Part 1: Install Libraries
!pip install qiskit qiskit-aer qiskit-ibm-runtime

# Part 2: Import and Run Adversarial Experiment (New Method)
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.sampler import SamplerV2 as Sampler
from qiskit import QuantumCircuit, transpile

print("\n--- Libraries imported successfully ---\n")

# --- Circuit Definitions (unchanged) ---
def create_bv_circuit(secret_string):
    num_qubits = len(secret_string) + 1
    qc = QuantumCircuit(num_qubits, len(secret_string))
    qc.h(range(num_qubits-1)); qc.x(num_qubits-1); qc.h(num_qubits-1); qc.barrier()
    for i, char in enumerate(reversed(secret_string)):
        if char == '1': qc.cx(i, num_qubits-1)
    qc.barrier(); qc.h(range(num_qubits-1)); qc.barrier()
    qc.measure(range(len(secret_string)), range(len(secret_string)))
    return qc

def create_crosstalk_hammer(num_qubits, repetitions):
    qc = QuantumCircuit(num_qubits)
    for _ in range(repetitions):
        for i in range(0, num_qubits - 1, 2): qc.cx(i, i+1)
        qc.rz(1.2, range(num_qubits)); qc.ry(0.8, range(num_qubits))
    return qc

secret_key = '101'

# --- Connect to IBM Service ---
service = QiskitRuntimeService(channel="ibm_cloud", token="KYLgcVygZxhhdMlHPbHXc71p64PE-Wgb1C7So3Qbfd_w", instance="crn:v1:bluemix:public:quantum-computing:us-east:a/c4b54a6a008d47f2afbe785476cdbd53:93ed1496-2c52-4df5-b0db-43c824e7a6b2::")
backend_real = service.backend("ibm_brisbane")
print(f"Successfully connected to backend: {backend_real.name}")

# --- NEW WORKFLOW: COMPOSE THEN TRANSPILE ---
# 1. Create the logical circuits
victim_circuit = create_bv_circuit(secret_key)
attacker_circuit = create_crosstalk_hammer(num_qubits=4, repetitions=20)

# 2. Compose them onto a larger logical circuit
# Victim on logical qubits 0-3, Attacker on logical qubits 4-7
combined_logical = QuantumCircuit(8, 3)
combined_logical.compose(victim_circuit, qubits=[0, 1, 2, 3], clbits=[0, 1, 2], inplace=True)
combined_logical.compose(attacker_circuit, qubits=[4, 5, 6, 7], inplace=True)
print("Logical circuits composed successfully.")

# 3. Define the mapping from logical to physical qubits
# We will map our 8 logical qubits to two adjacent groups of 4 physical qubits on the chip.
# For ibm_brisbane, physical qubits {8,9,10,11} are next to {12,13,14,15}.
initial_layout = [8, 9, 10, 11, 12, 13, 14, 15]

# 4. Transpile the single combined circuit with the desired layout
print("Transpiling the combined circuit for the hardware layout...")
combined_transpiled = transpile(combined_logical, backend=backend_real, initial_layout=initial_layout, optimization_level=3)
print("Circuit transpiled successfully.")

# --- Run the Adversarial Job using SamplerV2 ---
sampler = Sampler(mode=backend_real)
shots = 8192

print("\nSubmitting ADVERSARIAL job to real hardware...")
job_adversarial = sampler.run([combined_transpiled], shots=shots)
print(f"Job submitted with ID: {job_adversarial.job_id()}")
print("Waiting for job to complete...")

result_adversarial = job_adversarial.result()
pub_result = result_adversarial[0]
# The classical register is now associated with the combined circuit
counts_adversarial = pub_result.data.c.get_counts()
prob_dist_adversarial = {bitstring: count / shots for bitstring, count in counts_adversarial.items()}

print("\n✅ Adversarial job finished!")
print(f"Adversarial results (probabilities): {prob_dist_adversarial}")

if secret_key in prob_dist_adversarial:
    p_success_adversarial = prob_dist_adversarial[secret_key]
else:
    p_success_adversarial = 0

print(f"\nAdversarial Success Probability: {p_success_adversarial:.4f}")

# --- Final AER Calculation ---
p_success_baseline = 0.9065 # The value from our successful baseline run
aer = (p_success_baseline - p_success_adversarial) / (1 - p_success_baseline + 1e-9)
print(f"\nCalculated Adversarial Error Rate (AER): {aer:.2%}")


--- Libraries imported successfully ---

Successfully connected to backend: ibm_brisbane
Logical circuits composed successfully.
Transpiling the combined circuit for the hardware layout...
Circuit transpiled successfully.

Submitting ADVERSARIAL job to real hardware...
Job submitted with ID: d2um2gputc7s73aomfag
Waiting for job to complete...

✅ Adversarial job finished!
Adversarial results (probabilities): {'101': 0.8592529296875, '100': 0.0352783203125, '010': 0.0010986328125, '110': 0.0140380859375, '111': 0.0333251953125, '000': 0.0233154296875, '001': 0.0318603515625, '011': 0.0018310546875}

Adversarial Success Probability: 0.8593

Calculated Adversarial Error Rate (AER): 50.53%


In [20]:
#EXP-2 LCI
# Part 1: Install Libraries
!pip install qiskit qiskit-aer qiskit-ibm-runtime scikit-learn

# Part 2: Import and Run LCI Experiment
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.sampler import SamplerV2 as Sampler
from qiskit import QuantumCircuit, transpile
from sklearn.metrics import mutual_info_score
import numpy as np

print("\n--- Libraries imported successfully ---\n")

# --- Circuit Definitions ---
def create_qft_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits, num_qubits)
    for i in range(num_qubits):
        qc.h(i)
        for j in range(i + 1, num_qubits):
            qc.cp(np.pi / (2**(j - i)), j, i)
    for i in range(num_qubits // 2):
        qc.swap(i, num_qubits - 1 - i)
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

def create_vqe_ansatz_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits, num_qubits)
    for i in range(num_qubits):
        qc.ry(np.random.uniform(0, 2 * np.pi), i)
    for i in range(0, num_qubits - 1, 2):
        qc.cx(i, i+1)
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

def create_snooper_circuit(num_qubits):
    # A simple circuit that prepares a superposition and waits, making it sensitive to noise
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    qc.barrier()
    qc.delay(1000, unit='dt') # Wait for a short time
    qc.barrier()
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

victim_circuits = {
    "qft": create_qft_circuit(4),
    "vqe": create_vqe_ansatz_circuit(4)
}
snooper_circuit = create_snooper_circuit(4)

# --- Connect to IBM Service ---
service = QiskitRuntimeService(channel="ibm_cloud", token="KYLgcVygZxhhdMlHPbHXc71p64PE-Wgb1C7So3Qbfd_w", instance="crn:v1:bluemix:public:quantum-computing:us-east:a/c4b54a6a008d47f2afbe785476cdbd53:93ed1496-2c52-4df5-b0db-43c824e7a6b2::")
backend_real = service.backend("ibm_brisbane")
print(f"Successfully connected to backend: {backend_real.name}")

# --- Transpile and Run for each Victim ---
# Place snooper on physical qubits {8,9,10,11} and victim on adjacent {12,13,14,15}
snooper_layout = [8, 9, 10, 11]
victim_layout = [12, 13, 14, 15]
initial_layout = snooper_layout + victim_layout
shots = 8192
sampler = Sampler(mode=backend_real)

snooper_results = {}

for name, victim_qc in victim_circuits.items():
    print(f"\nRunning snooper next to victim '{name}'...")

    # Compose snooper and victim onto a single logical circuit
    combined_logical = QuantumCircuit(8, 4) # 8 qubits total, 4 clbits for snooper's result
    combined_logical.compose(snooper_circuit, qubits=[0, 1, 2, 3], clbits=[0, 1, 2, 3], inplace=True)
    combined_logical.compose(victim_qc, qubits=[4, 5, 6, 7], inplace=True)

    # Transpile the combined circuit
    combined_transpiled = transpile(combined_logical, backend=backend_real, initial_layout=initial_layout, optimization_level=3)

    # Run the job
    job = sampler.run([combined_transpiled], shots=shots)
    result = job.result()
    pub_result = result[0]

    # Store the snooper's measurement counts
    snooper_results[name] = pub_result.data.c.get_counts()
    print(f"Snooper results for '{name}': {snooper_results[name]}")

# --- Calculate LCI (using a proxy: classical Total Variation Distance) ---
# This measures how different the two probability distributions from the snooper are.
# A higher value means the attacker can more easily distinguish the victim circuits.
dist_qft = {int(k, 2): v / shots for k, v in snooper_results["qft"].items()}
dist_vqe = {int(k, 2): v / shots for k, v in snooper_results["vqe"].items()}

tvd = 0.5 * sum(abs(dist_qft.get(i, 0) - dist_vqe.get(i, 0)) for i in range(2**4))

print(f"\nDistinguishability (TVD) between snooper results: {tvd:.4f}")
print("A value closer to 1 means high information leakage; closer to 0 means low leakage.")


--- Libraries imported successfully ---

Successfully connected to backend: ibm_brisbane

Running snooper next to victim 'qft'...
Snooper results for 'qft': {'0111': 480, '0001': 386, '0011': 452, '0100': 699, '1001': 497, '1100': 703, '0101': 627, '1010': 495, '1110': 451, '1101': 775, '1111': 502, '0110': 407, '0000': 365, '0010': 434, '1000': 403, '1011': 516}

Running snooper next to victim 'vqe'...
Snooper results for 'vqe': {'0100': 2249, '1110': 726, '1100': 1186, '0111': 260, '0110': 1298, '1000': 767, '0101': 146, '1011': 95, '1010': 447, '0000': 433, '0011': 48, '1111': 139, '0010': 247, '1101': 67, '1001': 58, '0001': 26}

Distinguishability (TVD) between snooper results: 0.4432
A value closer to 1 means high information leakage; closer to 0 means low leakage.


In [23]:
#EXP-3 QDI
# --- QDI Step 1: Baseline Time ---
import time
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.sampler import SamplerV2 as Sampler
from qiskit import QuantumCircuit, transpile # Import transpile

# --- We only need a simple circuit for this timing test ---
def create_test_circuit():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure_all()
    return qc

test_circuit = create_test_circuit()

# --- Connect to IBM Service ---
service = QiskitRuntimeService(channel="ibm_cloud", token="KYLgcVygZxhhdMlHPbHXc71p64PE-Wgb1C7So3Qbfd_w", instance="crn:v1:bluemix:public:quantum-computing:us-east:a/c4b54a6a008d47f2afbe785476cdbd53:93ed1496-2c52-4df5-b0db-43c824e7a6b2::")
backend_real = service.backend("ibm_brisbane")
sampler = Sampler(mode=backend_real)
print(f"Connected to backend: {backend_real.name}")

# --- Transpile the circuit for the target backend ---
print("\nTranspiling test circuit for the target backend...")
transpiled_test_circuit = transpile(test_circuit, backend_real)
print("Test circuit transpiled successfully.")


# --- Time the job ---
print("\nSubmitting baseline timing job...")
start_time = time.time()
job_baseline = sampler.run([transpiled_test_circuit], shots=100) # Run the transpiled circuit
result = job_baseline.result() # This line waits for the job to complete
end_time = time.time()

baseline_duration = end_time - start_time
print(f"\n✅ Baseline job finished!")
print(f"Total time to completion (baseline): {baseline_duration:.2f} seconds")

Connected to backend: ibm_brisbane

Transpiling test circuit for the target backend...
Test circuit transpiled successfully.

Submitting baseline timing job...

✅ Baseline job finished!
Total time to completion (baseline): 4.22 seconds


In [26]:
#UNDER ATTACK
# --- QDI Step 2: Time Under Attack ---
import time
from qiskit import transpile # Import transpile

# --- Create the flood of small jobs ---
flood_circuit = QuantumCircuit(1, 1)
flood_circuit.measure(0, 0)
num_flood_jobs = 20

print(f"Submitting {num_flood_jobs} flood jobs to clog the queue...")
# NOTE: We are submitting simple circuits that are likely already in the backend's native gate set,
# so explicit transpilation for these small circuits might not be strictly necessary, but
# for robustness, one could transpile them too if they were more complex.
for i in range(num_flood_jobs):
    sampler.run([flood_circuit], shots=1) # Submit without waiting for the result
print("Flood jobs submitted.")

# --- Time the same test job again, but during the flood ---
# We need to recreate the test circuit and transpile it for this run
def create_test_circuit():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure_all()
    return qc

test_circuit_attack = create_test_circuit() # Create a new instance
print("\nTranspiling test circuit for the target backend (under attack)...")
# Transpile the test circuit for the attack run
transpiled_test_circuit_attack = transpile(test_circuit_attack, backend_real)
print("Test circuit transpiled successfully (under attack).")


print("\nSubmitting test job under attack conditions...")
start_time_attack = time.time()
# Run the transpiled test circuit under attack
job_attack = sampler.run([transpiled_test_circuit_attack], shots=100)
result_attack = job_attack.result()
end_time_attack = time.time()

attack_duration = end_time_attack - start_time_attack
print(f"\n✅ Attacked job finished!")
print(f"Total time to completion (under attack): {attack_duration:.2f} seconds")

# --- Final QDI Calculation ---
# Use the 'baseline_duration' variable from the previous cell
# Add a small epsilon to the denominator to avoid division by zero if baseline_duration is 0
qdi = ((attack_duration - baseline_duration) / (baseline_duration + 1e-9)) * 100
print(f"\nCalculated Queue Delay Inflation (QDI): {qdi:.2f}%")

Submitting 20 flood jobs to clog the queue...
Flood jobs submitted.

Transpiling test circuit for the target backend (under attack)...
Test circuit transpiled successfully (under attack).

Submitting test job under attack conditions...

✅ Attacked job finished!
Total time to completion (under attack): 4.71 seconds

Calculated Queue Delay Inflation (QDI): 11.64%


You now have the full set of quantitative results for your first provider:

AER (Integrity): 50.53%

LCI (Confidentiality): 0.4432 TVD

QDI (Availability): 11.64%

